# Global Coordinate Conversions

In [ ]:
import numpy as np
from gnss_lib_py.utils import coordinates as coord

ECEF coordintes for N positions can be converted to LLA and back by
using functions defined in `utils/coordinates.py`

In [ ]:
# Using the LLA coordinates the Aero/Astro department at Stanford University
x_lla = np.array([[37.427112], [-122.1764146], [16]])
print('Converted ECEF coordinates')
print(coord.geodetic_to_ecef(x_lla))

x_ecef = np.array([[-2700628], [-4292443], [3855152]])
print('Converted LLA coordinates')
print(coord.ecef_to_geodetic(x_ecef))

# Local NED Frame Conversions

A local North-East-Down (NED) frame of reference can be instantiated
by defining the point at the origin of the coordinate frame using either
its ECEF position or LLA (w.r.t the WGS-84 ellipsoid) coordinates

In [ ]:
# Instantiate using LLA coordinates
local_frame = coord.LocalCoord.from_geodetic(x_lla)
print('NED to ECEF conversion matrix for initialized local frame')
print(local_frame.ned_to_ecef_matrix)

local_frame = coord.LocalCoord.from_geodetic(x_ecef)
print('NED to ECEF conversion matrix for initialized local frame')
print(local_frame.ned_to_ecef_matrix)

Once initialized, the `local_frame` can be used to convert position
vectors to NED from ECEF or LLA and vice-versa

In [ ]:
ned = local_frame.ecef_to_ned(x_ecef)
print('The converted NED coordinates are')
print(ned)

ecef = local_frame.ned_to_ecef(ned)
print('The converted ECEF coordinates are')
print(ecef)

lla = local_frame.ned_to_geodetic(ned)
print('The converted LLA coordinates are')
print(lla)

The `local_frame` can also be used to convert free vectors in the NED
frame of reference to free vectors in the ECEF frame of reference

In [ ]:
v_vect = np.array([[1], [0], [0]])
v_ned = local_frame.ned_to_ecefv(v_vect)
print('The converted free vector in ECEF is')
print(v_ned)
v_ecef = local_frame.ecef_to_nedv(v_ned)
print('The converted free vector in NED is ')
print(v_ecef)

# Elevation and Aziumth from ECEF Positions

Find elevation and azimuth angle from receiver and satellite ECEF positions.

In [ ]:
from gnss_lib_py.utils.coordinates import ecef_to_el_az
from gnss_lib_py.parsers.android import AndroidDerived2022

# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2022/device_gnss.csv --quiet -O "device_gnss.csv"
navdata = AndroidDerived2022("device_gnss.csv")
navdata_subset = navdata.where("gps_millis",navdata["gps_millis",0]) # only use data from first timestep

To calculate the elevation and azimuth, pass in the receiver and satellites' ECEF positions.

In [ ]:
pos_sv_m = navdata_subset[["x_sv_m","y_sv_m","z_sv_m"]]
pos_rx_m = navdata_subset[["x_rx_m","y_rx_m","z_rx_m"],0].reshape(-1,1)

calculated_el_az = ecef_to_el_az(pos_rx_m,pos_sv_m)
truth_el_az = navdata_subset[["el_sv_deg","az_sv_deg"]]

We can now compare the calculated elevation and azimuth with their respective "truth" values included in the Google Decimeter Challenge 2022 dataset.

In [ ]:
for sat_idx in range(3):
    print(f"SV ID: {int(navdata_subset['sv_id',sat_idx])}")
    print(f"Calculated elevation: {calculated_el_az[0, sat_idx]}, Truth elevation: {truth_el_az[0, sat_idx]}")
    print(f"Calculated azimuth: {calculated_el_az[1, sat_idx]}, Truth azimuth: {truth_el_az[1, sat_idx]}")